In [1]:
import pandas as pd
import seaborn as sb

# 1 Select Data

In [2]:
cot_all = pd.read_excel("..\\..\\Datos\\working\\cotizaciones_all.xlsx")
neg_all = pd.read_excel("..\\..\\Datos\\working\\negocios_all.xlsx")

In [3]:
# cot_all.columns

In [4]:
cot_all.drop(['Remoto','Etapa','Tipo de Medio','Razon Social','Cargo',
              'Situacion Laboral','Antiguedad Laboral'],
            axis=1,inplace=True)

In [5]:
cot_mod = cot_all.copy()

In [6]:
# ruts de los clientes en negocio
neg_ids = neg_all['ID Cotizacion'].unique().tolist()

# cot_neg = cot_all.copy()
cot_mod['Negocio'] = cot_all['ID'].isin(neg_ids)
cot_mod[cot_mod['Negocio'] == True].shape

(2336, 35)

# 2 Clean Data

In [7]:
# variables a impiar y chequear

# Limpiar variable fecha de nacimiento de alguna manera
# Limpiar variable Sexo usando rutificador
# Limpar variables Direccion usando en lo posible el rutificador y otras fuentes
# Llenar mv en decuentos con 0

## 2.1 Rut
Arreglar Ruts, luego validarlos para aplicar data crawling en rutificador.
Si el rut esta mal escrito por ende no es valido, se prosigue a buscar el rut por el nombre y arreglar el rut.
Todo esto con el find de obtener el genero, direccion, latiyud y longitud del cotizante

### 2.1.1 Funciones
Funciones para tratar los ruts

In [8]:
def format_rut(x):
    rut = x.split('-')
    l_s = list(rut[0])
    r_l = l_s[::-1]
    new = []
    i = 0
    new.append(rut[1])
    new.append('-')
    for c in r_l:
        if i==3:
            new.append('.')
            i = 0
        new.append(c)
        i +=1
    rut = ''.join(new[::-1])
    return rut

In [9]:
import chile_rut
import re

def val_rut(x):
    x = str(x)
    if len(x) == 0:
        return False
    x = re.sub('[.|!@#$]', '', x)
    
    if len(x.split('-')) >= 2:
        return chile_rut.validate_rut(x)
    else:
        x = x[:-1] + '-' + x[-1]
        return chile_rut.validate_rut(x)

def clean_rut(x):
    x = str(x)
    if len(x) == 0:
        return False
    x = re.sub('[.|!@#$]', '', x)
    
    if len(x.split('-')) >= 2:
        return x
    else:
        x = x[:-1] + '-' + x[-1]
        return x
    
cot_mod['Clean Rut'] = [clean_rut(x) for x in cot_mod['RUT'].values]
cot_mod['Valid Rut'] = [val_rut(x) for x in cot_mod['Clean Rut'].values]


### 2.1.1 Insights
Revision de los ruts, mal formaciones, ruts malos que tienen negocios, etc

In [10]:
#Eliminar indice 1704 y arreglar rut max boris
print('Ruts que hicieron negocios pero no son validos')
cot_mod[(cot_mod['Valid Rut'] == False) & (cot_mod['Negocio'] == True)][['Nombre','RUT','Negocio','Valid Rut','Clean Rut']]

Ruts que hicieron negocios pero no son validos


,Nombre,RUT,Negocio,Valid Rut,Clean Rut
7390,max boris,14030274-6,True,False,14030274-6
7504,max boris,14030274-6,True,False,14030274-6
1704,Error,15516786-8,True,False,15516786-8


In [11]:
# ver que ruts invalidos pueden ser rescatados o arreglados buscando por sus nombres
print("Ruts catalogados como invalidos, buscar por nombres")
ruts_iv = cot_mod[(cot_mod['Valid Rut'] == False)] 
print(ruts_iv.shape)

ruts_invalidos = []
for group, frame in  ruts_iv.groupby('Clean Rut'):
    ruts_invalidos.append(str(group))

s_ruts = pd.Series(ruts_invalidos)
s_ruts.to_csv("..\\..\\Datos\\ruts_invalidos_scrap.csv")

# print(ruts_iv[['Nombre','Apellido 1','RUT','Clean Rut','Valid Rut']].head())

Ruts catalogados como invalidos, buscar por nombres
(69, 37)


In [12]:
print("Ruts arreglados, estaban mal escritos y luego fueron validos.")
df = cot_mod[(cot_mod['RUT'] != cot_mod['Clean Rut']) & (cot_mod['Valid Rut'] ==True)]
print(df.shape)
print(df[['Nombre','Apellido 1','Apellido 2','RUT','Clean Rut','Valid Rut']].head(10))

Ruts arreglados, estaban mal escritos y luego fueron validos.
(1, 37)
      Nombre Apellido 1 Apellido 2        RUT   Clean Rut  Valid Rut
8  Valentina  Fernandez          .  19483862k  19483862-k       True


In [13]:
print("Ruts arreglados, estaban mal escritos y luego fueron validos.")
df = cot_mod[(cot_mod['RUT'] != cot_mod['Clean Rut']) & (cot_mod['Valid Rut'] ==False)]
print(df.shape)
print(df[['Nombre','Apellido 1','Apellido 2','RUT','Clean Rut','Valid Rut']].head(10))

Ruts arreglados, estaban mal escritos y luego fueron validos.
(6, 37)
              Nombre Apellido 1 Apellido 2           RUT   Clean Rut  \
821         Giovanni     Flores          .  16.546.158-2  16546158-2   
841         Giovanni     Flores          .  16.546.158-2  16546158-2   
3716           Mario     Concha     Concha     156778804  15677880-4   
8943    Jose Edgardo    Caamaño   Chamorro       6180535    618053-5   
9099   Dinora  Nilda      Bravo      Cerda       9559524    955952-4   
9574  Anegelo Felipe      Godoy    Morales    |7321289-5   7321289-5   

      Valid Rut  
821       False  
841       False  
3716      False  
8943      False  
9099      False  
9574      False  


In [14]:
print("Ruts que estaban bien escritos.")
df = cot_mod[(cot_mod['RUT'] == cot_mod['Clean Rut'])]
print(df.shape)

Ruts que estaban bien escritos.
(16585, 37)


In [15]:
print("Ruts que estaban bien escritos y son validos.")
df = cot_mod[(cot_mod['RUT'] == cot_mod['Clean Rut']) &(cot_mod['Valid Rut'] ==True)]
print(df.shape)

ruts_validos = []
for group, frame in  df.groupby('Clean Rut'):
    ruts_validos.append(str(group))

# print(ruts_invalidos)
ruts_validos= [format_rut(x) for x in ruts_validos]
cot_mod['Format Rut'] = [format_rut(x) for x in cot_mod['Clean Rut'].values]
s_ruts = pd.Series(ruts_validos)
s_ruts.to_csv("..\\..\\ruts_validos_scrap.csv")

Ruts que estaban bien escritos y son validos.
(16522, 37)


In [16]:
print("Ruts que estaban bien escritos pero no son validos, buscar ruts por nombres.")
df = cot_mod[(cot_mod['RUT'] == cot_mod['Clean Rut']) &(cot_mod['Valid Rut'] ==False)]
print(df.shape)

Ruts que estaban bien escritos pero no son validos, buscar ruts por nombres.
(63, 38)


### 2.1.1 Ruts Scrapped

In [17]:
#Ruts y sus datos obtenidos de forma valida
ruts_scrapped = pd.read_csv('..\\..\\DataCrawling\\rut\\ruts_validos.csv')
ruts_scrapped['RUT'] = [x.replace(' ','') for x in ruts_scrapped['RUT'].values]
ruts_scrapped['RUT'] = [x.replace('K','k') for x in ruts_scrapped['RUT'].values]
print(ruts_scrapped.shape)
print(ruts_scrapped.head(2))

(6012, 5)
                                  Name           RUT Gender  \
0   SCHNEIDER BARRERA RICARDO ERNESTO   10.023.348-7   VAR    
1     MORALES ESPINOZA CLAUDIO ANDRES   10.020.262-k   VAR    

                             Adress               Comuna  
0                J  CISTERNAS 1612            LA SERENA   
1   ALEJANDRO SEXTO 2787LZO ARENAS    CONCEPCION CENTRO   


### 2.1.1 Ruts Scrapped merge a DF principal

In [18]:
# merge
cot_mod_cp = cot_mod.copy()
cot_mod_cp['Format Rut'] = [x.replace('K','k') for x in cot_mod_cp['Format Rut'].values]
cot_merge = pd.merge(cot_mod_cp, ruts_scrapped,how='left', left_on='Format Rut', right_on='RUT')
# print(cot_merge.columns)
columns = ['Format Rut','Valid Rut','Negocio','Nombre','Apellido 1','Name','Gender','Sexo',
           'Adress','Direccion', 'Region','Provincia', 'Comuna_x', 'Comuna_y']
cot_merge[columns][cot_merge['Sexo'].isnull()].shape

(1869, 14)

In [19]:

ruts_scrapped['RUT'] = [x.replace(' ','') for x in ruts_scrapped['RUT'].values]

# df = pd.merge(cot_mod_cp,ruts_scrapped,how='outer',left_on='Format Rut',right_on='RUT')
df = pd.merge(cot_mod_cp,ruts_scrapped,how='outer',left_on='Format Rut',right_on='RUT')


df.to_excel('..\\..\\Datos\\tmp.xlsx')

In [20]:
# len(df[df['Negocio'] == True]['RUT'].unique())

# 3 Construct Data

## 3.1 Nuevos Atributos

### 3.1.1 Edad
Variable construida a partir de la fecha de nacimiento

In [21]:
# Construir variable 'Edad' a partir de fecha de Nacimiento
import datetime
now = datetime.datetime.now().year

fechas_nacimiento = pd.to_datetime(cot_all['Fecha Nacimiento']).apply(lambda x: x.date()).tolist()

def anio_nac(x):
    try:
        year = int(x.year)
        dif = now - year
        if dif >= 18:
            return dif
        else:
            return 0
    except (ValueError,AttributeError):
        return 0
        
    return r

edad = [anio_nac(x) for x in fechas_nacimiento]
cot_mod['edad'] = edad



In [22]:
cot_mod = cot_mod.drop(['Fecha Nacimiento'], axis=1)

### 3.1.2 Descuento
Variable que indica si el cliente recibio descuento

In [23]:
# Construir 'descuento': binario : [True, False]
recibio_descuento = cot_all['Descuentos'].tolist()

def is_desc(x):
    try:
        desc = int(x)
        if desc > 0:
            return True
        else:
            return False
    except ValueError:
        return False

descuento = [is_desc(x) for x in recibio_descuento]
cot_mod['recibio_descuento'] = descuento
# descuento

### 3.1.3.Cantidad de Productos

In [24]:
# Construir 'productos': categorico : ['depto','casa','estacionamiento','bodega','Estudio']
productos = cot_all['Productos'].tolist()
depto = []; estacionamiento=[]; bodega = []; estudio = [];nan = 0
def get_products(x,word,numeric=False):
    w=0
    try:
        p =str(x)
        arr_p = x.split(',')
        for e in arr_p:
            if word in e:
                w+=1
#                 print(word + " :",e)
                break
            elif numeric and ('B' not in e or 'Est' not in e):
                try:
                    int(e)
#                     print(word + " :",e)
                    w+=1
                except ValueError:
                    pass
        return w
    except (ValueError,AttributeError):
        try:
            int(x)
            if numeric:
                return 1
            else:
                return 0
        except ValueError:
            return 0

cot_mod['#vivienda'] = [get_products(x,'T',numeric=True) for x in productos]
cot_mod['#bodega'] = [get_products(x,'Bod') for x in productos]
cot_mod['#estacionamiento'] = [get_products(x,'Est') for x in productos]
cot_mod['#estudio'] = [get_products(x,'Estudio') for x in productos]
cot_mod['#lan'] = [get_products(x,'lan') for x in productos]

In [25]:
print(cot_mod[['Productos','#vivienda','#bodega','#estacionamiento','#estudio','#lan']].head())
print(cot_mod[['Productos','#vivienda','#bodega','#estacionamiento','#estudio','#lan']].tail())

              Productos  #vivienda  #bodega  #estacionamiento  #estudio  #lan
0         TB-104,Est-65          1        0                 1         0     0
1  TA-302,Est-41,Bod-02          1        1                 1         0     0
2         TA-303,Est-01          1        0                 1         0     0
3  TA-302,Est-41,Bod-06          1        1                 1         0     0
4         TA-604,Est-01          1        0                 1         0     0
        Productos  #vivienda  #bodega  #estacionamiento  #estudio  #lan
2912  703,Est-003          1        0                 1         0     0
2913          912          1        0                 0         0     0
2914  Est-002,703          1        0                 1         0     0
2915  912,Est-002          1        0                 1         0     0
2916          912          1        0                 0         0     0


In [26]:
cot_mod = cot_mod.drop(['Productos'], axis=1)

### 3.1.4 Latitud y Longitud

In [27]:
from geopy.geocoders import Nominatim

def get_latlong(direccion):
    geolocator = Nominatim()
    direccion = "TUCAPEL 937 , CONCEPCION CENTRO"
    location = geolocator.geocode(direccion)
#     print(location.address)
    # Flatiron Building, 175, 5th Avenue, Flatiron, New York, NYC, New York, ...
    print((location.latitude, location.longitude))
    # (40.7410861, -73.9896297241625)
#     print(location.raw)

# get_latlong('')

# cot_mod['loc_lat'] = []

### 3.1.6 Region

In [28]:
cot_mod['Region'].unique()

array(['VIII Región del Bío-Bío', 'XIII Región Metropolitana de Santiago',
       'VII Región del Maule', 'II Región de Antofagasta',
       'IV Región de Coquimbo', 'X Región de Los Lagos',
       'XII Región de Magallanes y de La Antártica Chilena',
       'V Región de Valparaíso',
       'VI Región del Libertador General Bernardo O Higgins',
       'XIV Región de Los Ríos',
       'XI Región de Aysen del General Carlos Ibáñez del Campo',
       'I Región de Tarapacá', 'IX Región de La Araucanía',
       'III Región de Atacama', 'XV Region de Arica y Parinacota'],
      dtype=object)

In [29]:
cot_mod['region'] = cot_all['Region'].copy()
cot_mod['region'] = cot_mod['region'].replace(
    {'VIII Región del Bío-Bío':'Bio-Bio',
     'XIII Región Metropolitana de Santiago':'Metropolitana'}
)
cot_mod['region'] = cot_mod['region'].replace(
    ['II Región de Antofagasta',
       'IV Región de Coquimbo', 'VII Región del Maule',
       'X Región de Los Lagos',
       'V Región de Valparaíso',
       'VI Región del Libertador General Bernardo O Higgins',
       'XIV Región de Los Ríos',
       'XI Región de Aysen del General Carlos Ibáñez del Campo',
       'I Región de Tarapacá', 'IX Región de La Araucanía',
       'XII Región de Magallanes y de La Antártica Chilena',
       'XV Region de Arica y Parinacota', 'III Región de Atacama'],'Otro'
)

In [30]:
cot_mod['region'] = cot_mod['region'].astype('category')
cot_mod['region'].head()

0    Bio-Bio
1    Bio-Bio
2    Bio-Bio
3    Bio-Bio
4    Bio-Bio
Name: region, dtype: category
Categories (3, object): [Bio-Bio, Metropolitana, Otro]

In [31]:
cot_mod = cot_mod.drop(['Region'], axis=1)

### 3.1.7 Provincia
Esta variable indica si el cliente facilito el correo en alguna cotizacion

In [32]:
cot_all['Provincia'].unique()

array(['Concepción', 'Ñuble', 'Santiago', 'Talca', 'Biobío', 'Maipo',
       'Antofagasta', 'Limarí', 'Llanquihue', 'Magallanes', 'El Loa',
       'Isla de Pascua', 'Valparaíso', 'Cachapoal', 'Arauco',
       'Cordillera', 'Valdivia', 'Coyhaique', 'Linares', 'Osorno',
       'San Felipe de Aconcagua', 'Iquique', 'Cautín', 'Capitán Prat',
       'Aysén', 'Chacabuco', 'Copiapó', 'Melipilla', 'Malleco', 'Arica',
       'Elqui', 'Cauquenes', 'Colchagua', 'Última Esperanza', 'Curicó',
       'Chiloé', 'Choapa', 'Huasco'], dtype=object)

In [33]:
cot_mod['provincia'] = cot_all['Provincia'].copy()
cot_mod['provincia'] = cot_mod['provincia'].replace(
    {'Concepción':'Concepcion',
     'Santiago':'Santiago'}
)
cot_mod['provincia'] = cot_mod['provincia'].replace(
    ['Antofagasta', 'Limarí', 'Talca', 'Biobío', 'Ñuble',
       'Llanquihue','Maipo', 'Valparaíso', 'Cachapoal',
       'Arauco', 'Valdivia', 'Coyhaique', 'Linares', 'Osorno',
       'San Felipe de Aconcagua', 'Iquique', 'Cautín', 'Capitán Prat',
       'Aysén', 'Chacabuco', 'Melipilla', 'Magallanes', 'Malleco',
       'Arica ', 'Elqui', 'El Loa', 'Cauquenes', 'Colchagua', 'Valdivia ',
       'Copiapó', 'Última Esperanza', 'Cordillera', 'Curicó', 'Chiloé',
       'Choapa', 'Huasco','Arica','Isla de Pascua'],'Otro'
)

In [34]:
cot_mod['provincia'] = cot_mod['provincia'].astype('category')
cot_mod['provincia'].head()

0    Concepcion
1    Concepcion
2    Concepcion
3    Concepcion
4          Otro
Name: provincia, dtype: category
Categories (3, object): [Concepcion, Otro, Santiago]

In [35]:
cot_mod = cot_mod.drop(['Provincia'], axis=1)

### 3.1.8 Comuna
Esta variable indica si el cliente facilito el correo en alguna cotizacion

In [36]:
cot_all['Comuna'].unique()

array(['Concepción', 'Chillán', 'San Pedro de la Paz', 'Talcahuano',
       'La Florida', 'Hualpén', 'Santiago', 'Constitución', 'Los Ángeles',
       'Ñuñoa', 'Providencia', 'Penco', 'Tomé', 'Buin', 'Antofagasta',
       'Ovalle', 'Puerto Montt', 'Punta Arenas', 'Chiguayante',
       'El Bosque', 'Yungay', 'Florida', 'Calama', 'Lota', 'Coronel',
       'Isla de Pascua', 'Viña del Mar', 'Machalí', 'Arauco',
       'Las Condes', 'Vitacura', 'Pirque', 'Contulmo', 'Concón',
       'Santa Juana', 'Puerto Varas', 'Cañete', 'Huechuraba', 'Valdivia',
       'Coyhaique', 'Colbún', 'Osorno', 'San Felipe', 'Iquique',
       'Rancagua', 'Temuco', 'Cochrane', 'Chillán Viejo', 'Maipú',
       'Aysén', 'Villarrica', 'La Reina', 'Peñalolén', 'Estación Central',
       'Colina', 'Quilicura', 'Curanilahue', 'Copiapó', 'Algarrobo',
       'Talca', 'San Carlos', 'Mulchén', 'Coelemu', 'Angol', 'Laja',
       'Nacimiento', 'Arica', 'La Serena', 'Hualqui', 'San Miguel',
       'Pelluhue', 'Quillón', 'Valpar

In [37]:
cot_mod['comuna'] = cot_all['Comuna'].copy()

cot_mod['comuna'] = cot_mod['comuna'].replace(
    ['Ovalle', 'Constitución','Tomé', 'Chillán','Yungay',
     'Puerto Montt', 'Santiago', 'El Bosque', 'La Florida','Las Condes',
     'Ñuñoa', 'Providencia', 'Buin','Florida','Viña del Mar',
     'Machalí', 'Vitacura', 'Arauco', 'Contulmo', 'Concón',
     'Santa Juana', 'Puerto Varas', 'Cañete', 'Huechuraba', 'Valdivia',
     'Coyhaique', 'Colbún', 'Osorno', 'San Felipe', 'Iquique',
     'Rancagua', 'Lota', 'Temuco', 'Cochrane', 'Chillán Viejo', 'Maipú',
     'Aysén', 'Villarrica', 'La Reina', 'Peñalolén', 'Estación Central',
     'Colina', 'Quilicura', 'Curanilahue', 'Algarrobo', 'Talca',
     'San Carlos', 'Punta Arenas', 'Mulchén', 'Coelemu', 'Angol',
     'Laja', 'Nacimiento', 'Arica', 'La Serena', 'Hualqui',
     'San Miguel', 'Calama', 'Pelluhue', 'Quillón', 'Valparaíso',
     'Llanquihue', 'Los Álamos', 'Renca', 'Santa Cruz', 'Panguipulli',
     'Cabrero', 'Cerrillos', 'Pudahuel', 'Copiapó', 'Natales', 'Tirúa',
     'Cauquenes', 'La Cisterna', 'Yumbel', 'Ránquil', 'Padre Las Casas',
     'Las Cabras', 'Coquimbo', 'Lebu', 'San José de Maipo', 'San Ramón',
     'Curicó', 'Independencia', 'Lampa ', 'Castro', 'Punitaqui',
     'Conchalí', 'San Rosendo', 'Alto Hospicio', 'Illapel', 'Huasco',
     'Collipulli', 'Tucapel', 'Alto Biobío','Pirque','Lampa','Isla de Pascua','Antofagasta', 'Arica'],'Otro'
)

In [38]:
cot_mod['comuna'] = cot_mod['comuna'].astype('category')
cot_mod['comuna'].head()

0    Concepción
1    Concepción
2    Concepción
3    Concepción
4          Otro
Name: comuna, dtype: category
Categories (9, object): [Chiguayante, Concepción, Coronel, Hualpén, ..., Otro, Penco, San Pedro de la Paz, Talcahuano]

In [39]:
cot_mod = cot_mod.drop(['Comuna'], axis=1)

### 3.1.9 Atributos Provistos
Esta variable indica el numero de veces que el cliente ha hecho cotizaciones

In [40]:
cot_mod['is_nombre'] = ~cot_all['Nombre'].isnull()
cot_mod['is_apellido1'] = ~cot_all['Apellido 1'].isnull()
cot_mod['is_apellido2'] = ~cot_all['Apellido 2'].isnull()
cot_mod['is_nombrecompleto'] = ~cot_all['Nombre Completo'].isnull()
cot_mod['is_correo'] = ~cot_all['Correo Electronico'].isnull()
cot_mod['is_direccion'] = ~cot_all['Direccion'].isnull()
cot_mod['is_telefono'] = ~cot_all['Telefono'].isnull()
cot_mod['is_actividad'] = ~cot_all['Actividad'].isnull()
cot_mod['is_estado_civil'] = ~cot_all['Estado Civil'].isnull()
cot_mod['is_fnac'] = ~cot_all['Fecha Nacimiento'].isnull()
cot_mod['is_celular'] = ~cot_all['Celular'].isnull()


cot_mod = cot_mod.drop(['Profesión','Telefono','Correo Electronico','Telefono','Nombre','Apellido 1','Apellido 2','Nombre Completo'], axis=1)

### 3.1.9 Atributos Provistos
Esta variable indica el numero de veces que el cliente ha hecho cotizaciones

In [41]:
cot_mod['is_profesion'] = ~cot_all['Profesión'].isnull()

### 3.1.4 Cotizaciones por Proyecto

In [42]:
rut_dict = dict()

    
for index, row in cot_mod.iterrows():
    rut = row['Format Rut']
    pr = row['Proyecto']
    
    if rut not in rut_dict.keys():
        rut_dict[rut] = dict()
        rut_dict[rut]['Altos del Valle'] = 0
        rut_dict[rut]['Edificio Urban 1470'] = 0
        rut_dict[rut]['San Andres Del Valle'] = 0
        rut_dict[rut]['Edificio Mil610'] = 0
        rut_dict[rut]['Edificio Junge'] = 0
    
    rut_dict[rut][pr] +=1



rut_proyectos = pd.DataFrame.from_dict(rut_dict,orient='index').reset_index()
rut_proyectos.head()

# cot_mod = pd.merge(cot_mod_cp, rut_proyectos,how='left', left_on='Format Rut', right_on='index')

,index,Altos del Valle,Edificio Urban 1470,San Andres Del Valle,Edificio Mil610,Edificio Junge
0,000.000.000-0,0,1,1,0,0
1,1.462.558-5,0,0,0,1,0
2,10.000.513-1,0,0,0,0,2
3,10.000.522-0,0,0,0,0,2
4,10.005.853-7,0,0,0,0,2


In [43]:
cot_mod.columns

Index(['ID', 'Fecha Cotizacion', 'Medio', 'Proyecto', 'Presencial',
       'Jefe Comercial', 'Ejecutivo Comercial', 'Total Productos',
       'Descuentos', 'Valor Final Venta', 'RUT', 'Celular', 'Direccion',
       'Sexo', 'Estado Civil', 'Rango Edad', 'Tipo Cliente', 'Giro',
       'Nacionalidad', 'N° Grupo Familiar', 'Actividad', 'Empleador',
       'Negocio', 'Clean Rut', 'Valid Rut', 'Format Rut', 'edad',
       'recibio_descuento', '#vivienda', '#bodega', '#estacionamiento',
       '#estudio', '#lan', 'region', 'provincia', 'comuna', 'is_nombre',
       'is_apellido1', 'is_apellido2', 'is_nombrecompleto', 'is_correo',
       'is_direccion', 'is_telefono', 'is_actividad', 'is_estado_civil',
       'is_fnac', 'is_celular', 'is_profesion'],
      dtype='object')

### 3.1.9 Rango Edad
Esta variable indica el numero de veces que el cliente ha hecho cotizaciones

In [44]:
### Ver edades en cotizantes y cotizantes que llegan a negocios
rango_edad = cot_mod['Rango Edad'].tolist()
rango_edad_min = [];rango_edad_max = []

for e in rango_edad:
    try:
        edad = e.split('-')
        rango_edad_min.append(int(edad[0]))
        rango_edad_max.append(int(edad[1]))
    except (ValueError, AttributeError):
        rango_edad_min.append(0)
        rango_edad_max.append(0)
        
cot_mod['Edad Min'] = rango_edad_min
cot_mod['Edad Max'] = rango_edad_max

### 3.1.11 Negocio
Indica si el cliente siguio el proceso de cotizacion y lo transformo en un proceso de negocio

In [46]:
# ruts de los clientes en negocio
neg_ids = neg_all['ID Cotizacion'].unique().tolist()

# cot_neg = cot_all.copy()
cot_mod['Negocio'] = cot_all['ID'].isin(neg_ids)
cot_mod[cot_mod['Negocio'] == True].shape

(2336, 50)

### 3.1.12 N° Grupo Familiar


In [48]:
cot_mod['N° Grupo Familiar']=cot_mod['N° Grupo Familiar'].fillna('Sin Información')

In [49]:
def is_nrofam(x):
    if x == 'Sin Información':
        return False
    else:
        return True

cot_mod['is_nrofam'] = [ is_nrofam(x) for x in cot_mod['N° Grupo Familiar'] ]

In [50]:
cot_mod.to_excel('..\\..\\Datos\\cotizaciones_transformed.xlsx')

### 3.1.12 N° Grupo Familiar


### 3.2. Extraccion de Datos
Esta variable indica el numero de veces que el cliente ha hecho cotizaciones

In [93]:
#View data integration
iterator = 0
for group, frame in  cot_mod.groupby('Format Rut'):
    
#     print(frame['N° Grupo Familiar'].isnull())
    #Check , debe ir a data exploration
    arr = frame['Negocio'].unique()
    l = len(arr)
    if l > 1:
        if True in arr: 
            iterator +=1
            print(group)
#         print(l)

print(iterator)

10.011.197-7
10.016.093-5
10.025.981-8
10.033.390-2
10.034.279-0
10.038.900-2
10.051.640-3
10.068.204-4
10.069.788-2
10.137.843-8
10.168.822-4
10.184.906-6
10.237.540-8
10.299.543-0
10.305.957-7
10.312.153-1
10.344.199-4
10.352.115-7
10.362.175-5
10.398.483-1
10.424.643-5
10.444.048-7
10.455.079-7
10.462.028-0
10.479.661-3
10.481.733-5
10.506.051-3
10.516.915-9
10.540.790-4
10.552.598-2
10.555.698-5
10.556.056-7
10.577.927-5
10.588.792-2
10.592.310-4
10.615.011-7
10.623.814-6
10.627.157-7
10.637.991-2
10.662.350-3
10.665.303-8
10.686.102-1
10.686.979-0
10.731.863-1
10.748.815-4
10.780.375-0
10.784.972-6
10.791.004-2
10.792.684-4
10.801.522-5
10.842.892-9
10.858.552-8
10.872.319-k
10.877.439-8
10.931.772-1
10.932.782-4
10.939.389-4
10.964.274-6
10.972.232-4
10.973.168-4
10.973.879-4
10.994.400-9
11.239.681-0
11.276.929-3
11.307.631-3
11.352.033-7
11.403.245-k
11.412.541-5
11.448.637-k
11.457.624-7
11.494.360-6
11.569.893-1
11.572.060-0
11.582.754-5
11.583.365-0
11.679.025-4
11.682.141-9

17.614.409-2
17.615.106-4
17.615.151-k
17.615.964-2
17.616.672-k
17.616.718-1
17.616.873-0
17.617.568-0
17.617.876-0
17.688.758-3
17.699.393-6
17.843.930-8
17.844.156-6
17.896.973-0
17.955.885-8
17.969.751-3
17.981.732-2
17.981.856-6
17.981.986-4
18.067.323-7
18.068.648-7
18.106.463-3
18.110.927-0
18.111.004-k
18.142.459-1
18.143.038-9
18.143.325-6
18.144.306-5
18.144.457-6
18.158.539-0
18.290.803-7
18.342.664-8
18.386.444-0
18.419.959-9
18.815.516-2
18.962.075-6
19.373.375-1
19.454.989-k
19.520.484-5
21.279.977-7
22.082.217-6
22.120.134-5
23.056.541-4
24.023.866-7
24.137.745-8
24.431.769-3
24.436.319-9
24.448.087-K
24.730.975-6
24.832.760-k
25.285.213-1
25.296.872-5
25.363.085-K
25.406.101-8
25.434.747-7
25.526.438-9
4.733.447-0
4.984.981-8
4.988.511-3
5.008.689-5
5.174.535-3
5.301.735-5
5.351.630-0
5.371.822-1
5.411.048-0
5.488.405-2
5.516.302-2
5.520.792-5
5.564.009-2
5.611.760-1
5.671.312-3
5.679.524-3
5.759.267-2
5.802.314-0
5.997.717-2
6.049.569-6
6.376.294-6
6.411.534-0
6.437.91

In [91]:
True in [False, True]

True

### 3.1.10.Tiempo entre Cotizaciones
Esta variable indica el numero de veces que el cliente ha hecho cotizaciones

In [53]:
# Construir 'promedio_de_tiempo' entre cotizacones donde participa el mismo rut

In [76]:
cot_mod.columns

Index(['ID', 'Fecha Cotizacion', 'Medio', 'Proyecto', 'Presencial',
       'Jefe Comercial', 'Ejecutivo Comercial', 'Total Productos',
       'Descuentos', 'Valor Final Venta', 'RUT', 'Celular', 'Direccion',
       'Sexo', 'Estado Civil', 'Rango Edad', 'Tipo Cliente', 'Giro',
       'Nacionalidad', 'N° Grupo Familiar', 'Actividad', 'Empleador',
       'Negocio', 'Clean Rut', 'Valid Rut', 'Format Rut', 'edad',
       'recibio_descuento', '#vivienda', '#bodega', '#estacionamiento',
       '#estudio', '#lan', 'region', 'provincia', 'comuna', 'is_nombre',
       'is_apellido1', 'is_apellido2', 'is_nombrecompleto', 'is_correo',
       'is_direccion', 'is_telefono', 'is_actividad', 'is_estado_civil',
       'is_fnac', 'is_celular', 'is_profesion', 'Edad Min', 'Edad Max',
       'is_nrofam'],
      dtype='object')

In [77]:
len(cot_mod['Clean Rut'].unique())

6184

# 4.Integrate Data

In [94]:
# Construir '#cotizaciones' a partir del nro de filas donde participa el mismo rut
cot_ruts = pd.DataFrame()

nro_cot_depto, nro_cot_esta, nro_cot_estu, nro_cot_bod  = [], [], [], []

mean_cot_depto, mean_cot_esta, mean_cot_estu, mean_cot_bod = [], [], [], []

ruts, ruts_nro_cot, rut_reg, rut_prov , rut_com = [], [], [] , [], []

is_nombre, is_apellido1, is_appelido2, is_nombrecompleto, is_direccion = [], [], [], [], [] 
is_telefono, is_celular, is_fnac, is_nrofam, is_profesion = [], [], [], [], []

rut = []

max_rango_edad = []

nro_proyectos = []

negocio = []

tipo_cliente = []

sexo = []


iterator = 0
for group, frame in  cot_mod.groupby('Format Rut'):
    iterator += 1
    is_nombre.append(frame['is_nombre'].tolist()[0])
    is_apellido1.append(frame['is_apellido1'].tolist()[0])
    is_appelido2.append(frame['is_apellido2'].tolist()[0])
    is_nombrecompleto.append(frame['is_nombrecompleto'].tolist()[0])
    is_direccion.append(frame['is_direccion'].tolist()[0])
    is_telefono.append(frame['is_telefono'].tolist()[0])
    is_celular.append(frame['is_celular'].tolist()[0])
    is_fnac.append(frame['is_fnac'].tolist()[0])
    is_profesion.append(frame['is_profesion'].tolist()[0])
    is_nrofam.append(frame['is_nrofam'].tolist()[0])
    
    rut.append(group)
    sexo.append(frame['Sexo'].tolist()[0])
    
    tipo_cliente.append(frame['Tipo Cliente'].tolist()[0])
    
    max_rango_edad.append(frame['Edad Max'].tolist()[0])

    nro_proyectos.append(len(frame['Proyecto'].unique()))
    
    arr = frame['Negocio'].tolist()
    if True in arr:   
        negocio.append(True)
    else:
        negocio.append(False)
    
    ruts.append(group)
    rut_reg.append(frame['region'].tolist()[0])
    rut_prov.append(frame['provincia'].tolist()[0])
    rut_com.append(frame['comuna'].tolist()[0])
    
    nro_cot_depto.append(frame['#vivienda'].sum())
    nro_cot_bod.append(frame['#bodega'].sum())
    nro_cot_esta.append(frame['#estacionamiento'].sum())
    nro_cot_estu.append(frame['#estudio'].sum())
    
    mean_cot_depto.append(frame['#vivienda'].mean())
    mean_cot_bod.append(frame['#bodega'].mean())
    mean_cot_esta.append(frame['#estacionamiento'].mean())
    mean_cot_estu.append(frame['#estudio'].mean())
    ruts_nro_cot.append(frame['ID'].count())
    
print(iterator)

6184


In [95]:
list_cols = [ruts,rut_reg,rut_prov,rut_com,nro_proyectos,nro_cot_bod, nro_cot_depto, nro_cot_esta, nro_cot_estu, mean_cot_bod,
             mean_cot_depto,mean_cot_esta,mean_cot_estu, negocio,is_nombre,
             is_apellido1,is_appelido2,is_nombrecompleto,is_direccion,is_telefono,
             is_celular,is_fnac,max_rango_edad,tipo_cliente,sexo, is_nrofam, is_profesion,rut]
list_labels = ['rut','loc_region','loc_provincia','loc_comuna','nro_proyectos','nro_cot_bod', 'nro_cot_depto', 'nro_cot_esta', 'nro_cot_estu', 'mean_cot_bod',
             'mean_cot_depto','mean_cot_esta','mean_cot_estu','negocio',
            'is_nombre','is_apellido1','is_appelido2','is_nombrecompleto','is_direccion','is_telefono',
             'is_celular','is_fnac','max_rango_edad','tipo_cliente','sexo','is_nrofam','is_profesion','rut']

zipped = list(zip(list_labels,list_cols))
data = dict(zipped)
personas = pd.DataFrame(data)
print(personas.shape)
personas.head()

(6184, 27)


,is_apellido1,is_appelido2,is_celular,is_direccion,is_fnac,is_nombre,is_nombrecompleto,is_nrofam,is_profesion,is_telefono,...,mean_cot_estu,negocio,nro_cot_bod,nro_cot_depto,nro_cot_esta,nro_cot_estu,nro_proyectos,rut,sexo,tipo_cliente
0,True,True,True,False,True,True,True,False,True,True,...,0.0,False,1,2,1,0,2,000.000.000-0,Femenino,Natural
1,True,True,True,False,False,True,True,False,False,True,...,0.0,False,0,1,1,0,1,1.462.558-5,Masculino,Natural
2,True,True,True,False,False,True,True,False,False,False,...,1.0,False,2,2,2,2,1,10.000.513-1,Masculino,Natural
3,True,True,True,True,False,True,True,True,True,False,...,0.0,True,2,2,2,0,1,10.000.522-0,Masculino,Natural
4,True,True,True,False,False,True,True,False,False,False,...,0.0,False,2,2,2,0,1,10.005.853-7,NaN,Natural


In [96]:
print(len(personas['rut'].unique()))

6184


In [97]:
len(personas[personas.negocio == True])

1101

In [98]:
personas.columns

Index(['is_apellido1', 'is_appelido2', 'is_celular', 'is_direccion', 'is_fnac',
       'is_nombre', 'is_nombrecompleto', 'is_nrofam', 'is_profesion',
       'is_telefono', 'loc_comuna', 'loc_provincia', 'loc_region',
       'max_rango_edad', 'mean_cot_bod', 'mean_cot_depto', 'mean_cot_esta',
       'mean_cot_estu', 'negocio', 'nro_cot_bod', 'nro_cot_depto',
       'nro_cot_esta', 'nro_cot_estu', 'nro_proyectos', 'rut', 'sexo',
       'tipo_cliente'],
      dtype='object')

In [99]:
personas = pd.merge(personas, rut_proyectos, how = 'inner',left_on='rut',right_on='index')
personas = personas.drop(['index'],axis=1)

In [57]:
# personas.columns

In [58]:
# personas.info()

In [59]:
# ruts_scrapped.info()

In [100]:
personas = pd.merge(personas,ruts_scrapped,how='outer',left_on='rut',right_on='RUT')


In [101]:
personas = personas.drop(['Name','Adress','Comuna','RUT'],axis=1)

In [102]:
# personas.info()

In [103]:
sb.factorplot(x='sexo', kind='count', data=personas);

In [104]:
def clean_sex(x,y):
#     print(x)
    x = str(x)
    x = x.replace(' ','')
    if x == 'VAR':
        return 'Masculino'
    elif x == 'MUJ':
        return 'Femenino'
    else:
        return y

personas['sexo'] = [clean_sex(x,y) for x,y in zip(personas['Gender'].values,personas['sexo'].values)]

In [105]:
personas.sexo = personas.sexo.fillna('Sin Información')
sb.factorplot(x='sexo', kind='count', data=personas);

In [106]:
personas = personas.drop(['Gender'],axis=1)

In [107]:
print(personas.tipo_cliente.isnull().sum())
personas.tipo_cliente = personas.tipo_cliente.fillna('Natural')

87


# 5.Format Data

In [108]:
personas['loc_comuna'] = personas['loc_comuna'].astype('category')
personas['loc_provincia'] = personas['loc_provincia'].astype('category')
personas['loc_region'] = personas['loc_region'].astype('category')
personas['tipo_cliente'] = personas['tipo_cliente'].astype('category')
personas['sexo'] = personas['sexo'].astype('category')
personas = personas.dropna()

In [111]:
personas.to_csv('..\\..\\Datos\\working\\personas_cotizacion2.csv')
personas.to_excel('..\\..\\Datos\\working\\personas_cotizacion2.xlsx')

In [70]:
# personas.info()
# g = sb.factorplot(x='sexo', col='proyecto', kind='count', data=personas);